In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
from collections import Counter
import os
import re
from os import listdir
from os.path import isfile, join
import sklearn
import nltk
from nltk.corpus import stopwords 

from sklearn.linear_model import LogisticRegression


In [2]:
INPUT_DIR = '../data/input/groundtruth/'
OUTPUT_DIR = '../data/input/groundtruth/'

CORPUS_DIR = os.path.join(OUTPUT_DIR, 'corpus')
VECTORIZED_DIR = os.path.join(OUTPUT_DIR, 'vectorized_trainset')

text_col_header = 'text'
label_col_header = 'label'

### Reading test and train data to keep them ready for testing

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# Read Train data
ip_train_file = os.path.join(INPUT_DIR, 'speechact_augmented_train.csv')
df_train = pd.read_csv(ip_train_file)
df_train = df_train.astype({text_col_header: str, label_col_header: int})

# Read Test data
ip_test_file = os.path.join(INPUT_DIR, 'speechact_test.csv')
df_test = pd.read_csv(ip_test_file)
df_test = df_test.astype({text_col_header: str, label_col_header: int})

In [4]:
print(df_train.shape, df_test.shape)

(4268921, 2) (9275, 2)


# Logistic Regression

Requirements
<li> Input - Vectorized Data train </li>
<li> Input - Vectorized Label train </li>

### Counter Vector with stopwords

In [5]:
# Read the vectorized train data and train label
vectorized_train_data_file = os.path.join(VECTORIZED_DIR, 'aug_train_data_countvector_stopword.pkl')
with open(vectorized_train_data_file, 'rb') as f_ip:
    data_train = pkl.load(f_ip)
    
train_label_file = os.path.join(VECTORIZED_DIR, 'aug_train_label_countvector_stopword.pkl')
with open(train_label_file, 'rb') as f_ip:
    label_train = pkl.load(f_ip)

In [7]:
print(data_train.shape, label_train.shape)

(4268921, 61054) (4268921, 1)


#### Testing

In [10]:
# Read the vectorizer to transform the test data
vector_file = os.path.join(CORPUS_DIR, 'aug_vector_countvector_stopword.pkl')
with open(vector_file, 'rb') as f_ip:
    vectorizer = pkl.load(f_ip)

data_test = vectorizer.transform(df_test[text_col_header])

In [11]:
# Run Logistic Regression
estimator = LogisticRegression(max_iter=1000)
estimator.fit(data_train, np.ravel(label_train, order='C'))
predictions_test = estimator.predict(data_test)

#### Performance metrics

In [12]:
from sklearn.metrics import f1_score

## Accuracy Measure
print('Train Accuracy', estimator.score(data_train, df_train[label_col_header]))
print('Test Accuracy', estimator.score(data_test, df_test[label_col_header]))

# F1
f1_measure = f1_score(df_test[label_col_header], predictions_test, average='macro')
print('F1 macro Score: ', f1_measure)

# Classification report
from sklearn.metrics import classification_report
print(classification_report(df_test[label_col_header], predictions_test))

Train Accuracy 0.9971245661374385
Test Accuracy 0.9236657681940701
F1 macro Score:  0.9228482386941913
              precision    recall  f1-score   support

           0       0.94      0.92      0.93      3911
           1       0.92      0.92      0.92      2908
           2       0.90      0.93      0.92      2456

    accuracy                           0.92      9275
   macro avg       0.92      0.92      0.92      9275
weighted avg       0.92      0.92      0.92      9275



### Counter Vector with no stopwords

In [47]:
# Read the vectorized train data and train label
vectorized_train_data_file = os.path.join(VECTORIZED_DIR, 'train_data_countvector_nostopword.pkl')
with open(vectorized_train_data_file, 'rb') as f_ip:
    data_train = pkl.load(f_ip)
    
train_label_file = os.path.join(VECTORIZED_DIR, 'train_label_countvector_nostopword.pkl')
with open(train_label_file, 'rb') as f_ip:
    label_train = pkl.load(f_ip)

#### Testing

In [48]:
# Read the vectorizer to transform the test data
vector_file = os.path.join(CORPUS_DIR, 'vector_countvector_nostopword.pkl')
with open(vector_file, 'rb') as f_ip:
    vectorizer = pkl.load(f_ip)

data_test = vectorizer.transform(df_test[text_col_header])

In [49]:
# Run Logistic Regression
estimator = LogisticRegression(max_iter=1000)
estimator.fit(data_train, np.ravel(label_train, order='C'))
predictions_test = estimator.predict(data_test)

#### Performance metrics

In [50]:
from sklearn.metrics import f1_score

## Accuracy Measure
print('Train Accuracy', estimator.score(data_train, df_train[label_col_header]))
print('Test Accuracy', estimator.score(data_test, df_test[label_col_header]))

# F1
f1_measure = f1_score(df_test[label_col_header], predictions_test, average='macro')
print('F1 macro Score: ', f1_measure)

# Classification report
from sklearn.metrics import classification_report
print(classification_report(df_test[label_col_header], predictions_test))

Train Accuracy 0.9605380198927195
Test Accuracy 0.8659838274932614
F1 macro Score:  0.8661387567725706
              precision    recall  f1-score   support

           0       0.91      0.87      0.89      3911
           1       0.79      0.85      0.82      2908
           2       0.91      0.88      0.89      2456

    accuracy                           0.87      9275
   macro avg       0.87      0.87      0.87      9275
weighted avg       0.87      0.87      0.87      9275



### Counter Vector with minimum occurences of words (min_occurences=2)

In [51]:
# Read the vectorized train data and train label
vectorized_train_data_file = os.path.join(VECTORIZED_DIR, 'train_data_countvector_minoccurences_2.pkl')
with open(vectorized_train_data_file, 'rb') as f_ip:
    data_train = pkl.load(f_ip)
    
train_label_file = os.path.join(VECTORIZED_DIR, 'train_label_countvector_minoccurences_2.pkl')
with open(train_label_file, 'rb') as f_ip:
    label_train = pkl.load(f_ip)

#### Testing

In [52]:
# Read the vectorizer to transform the test data
vector_file = os.path.join(CORPUS_DIR, 'vector_countvector_minoccurences_2.pkl')
with open(vector_file, 'rb') as f_ip:
    vectorizer = pkl.load(f_ip)

data_test = vectorizer.transform(df_test[text_col_header])

In [53]:
# Run Logistic Regression
estimator = LogisticRegression(max_iter=1000)
estimator.fit(data_train, np.ravel(label_train, order='C'))
predictions_test = estimator.predict(data_test)

#### Performance metrics

In [54]:
from sklearn.metrics import f1_score

## Accuracy Measure
print('Train Accuracy', estimator.score(data_train, df_train[label_col_header]))
print('Test Accuracy', estimator.score(data_test, df_test[label_col_header]))

# F1
f1_measure = f1_score(df_test[label_col_header], predictions_test, average='macro')
print('F1 macro Score: ', f1_measure)

# Classification report
from sklearn.metrics import classification_report
print(classification_report(df_test[label_col_header], predictions_test))

Train Accuracy 0.9453624086902612
Test Accuracy 0.8636118598382749
F1 macro Score:  0.863375437946586
              precision    recall  f1-score   support

           0       0.91      0.87      0.89      3911
           1       0.78      0.85      0.81      2908
           2       0.90      0.87      0.89      2456

    accuracy                           0.86      9275
   macro avg       0.86      0.86      0.86      9275
weighted avg       0.87      0.86      0.86      9275

